# CS 445 Course Project

## Setup

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.spatial import Delaunay

In [ ]:
# #Brayden
datadir = "/Users/braydenturner/projects/MCS/cs445/course_project/" 

In [ ]:
# #Caleb
# datadir = "./CS445_CourseProject"

In [ ]:
# %matplotlib notebook
%matplotlib widget

# Load Images

In [ ]:
class Image:
    def __init__(self, path):
        self.image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB).astype('double') / 255.0

In [ ]:
img1 = Image('images/male.jpg') 
img2 = Image('images/female.jpg')

## Point Correspondence

In [ ]:
def specify_corresponding_points(img, clicked = None):
    '''
    img: input image
    clicked: ??
    return: clicked: Array of shape 2xP (number of points clicked)
            clicked[0]: array of length P containing the x coordinates
            clicked[1]: array of length P contraining y coordinates
    '''
    fig = plt.figure()
    if clicked:
        num = len(clicked[0])
        fig.set_label(f'Choose {num} corresponding points on image. Press q to quit')
    else:
        fig.set_label('Choose points on image. Press q to quit')
    plt.axis('off')
    plt.imshow(img)
    xs = []
    ys = []
    clicked = [] #CONCERN: Do we want this to be nested in the if clicked statement? 
#                          #or is the goal to reset the clicked array?

    def on_mouse_pressed(event):
        x = event.xdata
        y = event.ydata
        xs.append(x)
        ys.append(y)
        
        counter = len(xs)
        
        plt.plot(x, y, "r+")
        plt.text(x + 5, y + 5, counter, color="red")

    def onclose(event):
        clicked.append(xs)
        clicked.append(ys)
        
    def key_press(event):
        if event.key == 'q':
            print("Quitting")
            clicked.append(xs)
            clicked.append(ys)
            plt.close(fig)
            
        
        
    # Create an hard reference to the callback not to be cleared by the garbage
    # collector
    fig.canvas.mpl_connect('button_press_event', on_mouse_pressed)
    fig.canvas.mpl_connect('close_event', onclose)
    fig.canvas.mpl_connect('key_press_event', key_press)
    return clicked

In [ ]:
img1.pts = specify_corresponding_points(img1.image)

In [ ]:
img2.pts = specify_corresponding_points(img2.image, img1.pts)

In [ ]:
# Assert we have equal number of points
assert len(img1.pts) == len(img2.pts) and len(img1.pts[0]) == len(img2.pts[0])  and len(img1.pts[1]) == len(img2.pts[1])

## Triangulation Mesh

In [ ]:
# convert to list of (x,y) touples for each point for use in scipy.spatial.Delaunay()
def toPtArray(pts, h, w):
    numPts = len(pts[0])
    points = np.zeros((numPts + 4, 2)) # add 4 points for 4 corners
    print(numPts)
    for i in range(len(pts[1])):
        points[i][0] = pts[0][i]
        points[i][1] = pts[1][i]

    #Add the 4 corners
    points[numPts][0] = 0
    points[numPts][1] = 0
    points[numPts+1][0] = 0
    points[numPts+1][1] = h
    points[numPts+2][0] = w
    points[numPts+2][1] = 0
    points[numPts+3][0] = w
    points[numPts+3][1] = h
    return points

In [ ]:
# print(img1.pts)

In [ ]:
pts1 = toPtArray(img1.pts, img1.image.shape[0], img1.image.shape[1])
pts2 = toPtArray(img2.pts, img2.image.shape[0], img2.image.shape[1])


In [ ]:
tri1 = Delaunay(pts1)

# tri2 = Delaunay(pts2)

In [ ]:
fig1 = plt.figure()
plt.imshow(img1.image)
plt.triplot(pts1[:,0], pts1[:,1], tri1.simplices)
plt.plot(pts1[:,0], pts1[:,1], 'o')
plt.axis('off')
plt.show()

The pts in pts1 and pts2 will be matched up so long as when clicking the points they are tagged in corresponding order. By simply taking the Delaunay of the first image and using those same simplices with the points for imag 2 we have a matching triangle mesh for both images. 

In [ ]:
fig2 = plt.figure()
plt.imshow(img2.image)
#only change here is changed tri2 to tri1 below in order to get matching pt-triangle meshes.
plt.triplot(pts2[:,0], pts2[:,1], tri1.simplices)
plt.plot(pts2[:,0], pts2[:,1], 'o')
plt.axis('off')
plt.show()

In [ ]:
pts1.shape

The different attributes of the Delaunay can be found here: 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.Delaunay.html
What we still need to do is create some sort of correlation between the triangles of one and the triangles of the other according to Lect. 6.1.3 time: 4:39
That is probably easiest if we choose images with high correspondance to start 

Lecture notes (Apparently everyone is stealing the same stuff from Efros): 
https://www.seas.upenn.edu/~cse399b/Lectures/CSE399b-07-triangle.pdf

## Linear Interpolation

Equation for linear interpolation: P + t * (Q-P) : 0 < t < 1

In [ ]:
def lin_intepolate(pts1, pts2, n_steps):
    '''
    Takes pt locations for image1 and image2, and number of steps desired and 
    returns an array containing each point at each step in the linear interpolation
    
inputs:
    pts1:     array- size: num_points x 2 - array containing 2D locs. starting point of each linear interpolation
    pts2:     array- size: num_points x 2 - array containing 2D locs. end point of each linear interpolation
    n_steps:  int: indicating number of steps along linear path for each point
return: 
    l_interp: array- size: n_steps x num_points x 2 
    [step_number, pt_number, x or y]
    '''
    assert n_steps > 1
    num_pts, _ = pts1.shape
    l_interp = np.zeros((n_steps, num_pts,2))
    for j in range(num_pts):
        for xy in range(2):
            p1 = pts1[j,xy]
            p2 = pts2[j,xy]
            for i in range(n_steps):
                l_interp[i,j,xy] = p1 + (p2 - p1) * (i / (n_steps-1))
                
            
    return l_interp

In [ ]:
l = lin_intepolate(pts1, pts2, 5)

In [ ]:
test1 = np.arange(0,20).reshape(10,2)
print(test1)
test2 = np.arange(2,22).reshape(10,2)
print(test2)


In [ ]:
lin_intepolate(test1,test2, 2)

In [ ]:
lin_intepolate(test1,test2, 3)

In [ ]:
lin_intepolate(test1,test2, 4)

In [ ]:
lin_intepolate(test1,test2, 5)

## Affine Transformation

## Tie It All Together